In [1]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from normalsamplinglayer import NormalSamplingLayer
import numpy.random as rnd

from variationalautoencoder import VariationalAutoencoder
from joel_autoencoder import Autoencoder

ModuleNotFoundError: No module named 'autoencoder'

In [ ]:
# Global variables
latent_dim = 4

## Setup data
Load and preprocess data

After this section we will have the two variables *mnist_digits* and *mnist_labels*. They are both numpy arrays

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
mnist_labels = np.concatenate([y_train, y_test], axis=0)


## Setup autoencoders
Define the encoder and decoder which are then the defining features of our variational encoder and decoder

Define the encoders and decoders. The code is simply repeated for the autoencoder and the variational autoencoder. The only difference is that there is a parallell layer in the encoder for the variational autoencoder which defindes the variacnes.

In [ ]:
# Autoencoder encoder
encoder_input = keras.Input(shape = mnist_digits[0].shape)
net = encoder_input
net = keras.layers.Conv2D(32,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2D(64,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Flatten()(net)
net = keras.layers.Dense(16, activation = 'relu')(net)
z = keras.layers.Dense(latent_dim, name = 'z_mean')(net)
encoder_auto = keras.Model(encoder_input, z, name = 'encoder')

# Variational autoencoder encoder
encoder_input = keras.Input(shape = mnist_digits[0].shape)
net = encoder_input
net = keras.layers.Conv2D(32,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2D(64,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Flatten()(net)
net = keras.layers.Dense(16, activation = 'relu')(net)
z_mean = keras.layers.Dense(latent_dim, name = 'z_mean')(net)
z_log_var = keras.layers.Dense(latent_dim, name = 'z_log_var')(net)
z = NormalSamplingLayer()([z_mean, z_log_var])
encoder_var = keras.Model(encoder_input, [z_mean, z_log_var, z], name = 'encoder')

# Autoencoder decoder
decoder_input = keras.Input(shape = (latent_dim,))
net = decoder_input
net = keras.layers.Dense(7*7*64, activation = "relu")(net)
net = keras.layers.Reshape((7,7,64))(net)
net = keras.layers.Conv2DTranspose(64,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2DTranspose(32,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2DTranspose(1,3, activation = 'sigmoid', padding = 'same')(net)
decoder_output = net
decoder_auto = keras.Model(decoder_input, decoder_output, name = 'decoder')

# Variational autoencoder decoder
decoder_input = keras.Input(shape = (latent_dim,))
net = decoder_input
net = keras.layers.Dense(7*7*64, activation = "relu")(net)
net = keras.layers.Reshape((7,7,64))(net)
net = keras.layers.Conv2DTranspose(64,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2DTranspose(32,3, activation = 'relu', strides = 2, padding = 'same')(net)
net = keras.layers.Conv2DTranspose(1,3, activation = 'sigmoid', padding = 'same')(net)
decoder_output = net
decoder_var = keras.Model(decoder_input, decoder_output, name = 'decoder')



## Set up the autoencoder instances

In [ ]:
ae = Autoencoder(encoder_auto, decoder_auto)
va = VariationalAutoencoder(encoder_var, decoder_var)
ae.compile(optimizer=keras.optimizers.Adam())
va.compile(optimizer=keras.optimizers.Adam())

In [ ]:
ae.fit(mnist_digits,
      epochs = 20,
      batch_size = 256)

va.fit(mnist_digits,
      epochs = 20,
      batch_size = 256)

Epoch 1/20
274/274 [==============================] - 60s 213ms/step - Loss: 269.2907
Epoch 2/20
274/274 [==============================] - 67s 245ms/step - Loss: 141.2929
Epoch 3/20
274/274 [==============================] - 73s 268ms/step - Loss: 129.7478
Epoch 4/20
274/274 [==============================] - 67s 245ms/step - Loss: 125.5950
Epoch 5/20
274/274 [==============================] - 62s 226ms/step - Loss: 122.9442
Epoch 6/20
274/274 [==============================] - 67s 246ms/step - Loss: 121.6892
Epoch 7/20
274/274 [==============================] - 63s 230ms/step - Loss: 119.8369
Epoch 8/20
274/274 [==============================] - 64s 234ms/step - Loss: 118.8873
Epoch 9/20
274/274 [==============================] - 62s 225ms/step - Loss: 117.9139
Epoch 10/20
274/274 [==============================] - 68s 247ms/step - Loss: 117.1101
Epoch 11/20
274/274 [==============================] - 64s 232ms/step - Loss: 116.8547
Epoch 12/20
274/274 [==============================]

## Investigate models


### Project the latent space

In [ ]:
# import projection objects
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA


In [ ]:
# Predict the latent points for each input data
ae_mean = ae.encoder.predict(mnist_digits)
va_mean, z_var, va_z = va.encoder.predict(mnist_digits)

In [ ]:
# Project the latent point according to the t-SNE algorithm
ae_tsne = TSNE().fit_transform(ae_mean)
va_tsne = TSNE().fit_transform(va_mean)

In [ ]:
# Project the latent point according to PCA and caluclate the principal axes
pca_ae = PCA(n_components = 2).fit(ae_mean)
ae_pca = pca_ae.transform(ae_mean)
ae_vectors = (pca_ae.components_.T *3*np.sqrt(pca_ae.explained_variance_)).T
ae_origin = np.mean(ae_mean, axis = 0)

pca_va = PCA(n_components = 2).fit(va_mean)
va_pca = pca_va.transform(va_mean)
va_vectors = (pca_va.components_.T *3*np.sqrt(pca_va.explained_variance_)).T
va_origin = np.mean(va_mean, axis = 0)

### Run Gui:s

### Sample scatter plot
This visualization shows a 2D representation of the latent space and where each of the input point has been projected. This projection is done previously and is usually a PCA or t-SNE. Note that only one of the figures can work interactily at once.

In [ ]:
from sampleScatterGUI import SampleScatterGUI
from latentplanegui import LatentPlaneGUI
from latentinterpolationmosaic import LatentInterpolationMosaic

### Sample Scatter GUI

In [ ]:
# PCA/t-SNE of the autoencoder
#SampleScatterGUI(ae_pca, mnist_labels, mnist_digits)
SampleScatterGUI(ae_tsne, mnist_labels, mnist_digits)


In [ ]:
# PCA/t-SNE of the Variational Autoencoder
#SampleScatterGUI(va_pca, mnist_labels, mnist_digits)
SampleScatterGUI(va_tsne, mnist_labels, mnist_digits)

### Latent Plane GUI

In [ ]:
LatentPlaneGUI(ae.decoder,latent_vectors = ae_vectors, latent_origin = ae_origin)

In [ ]:
LatentPlaneGUI(va.decoder,latent_vectors = va_vectors, latent_origin = va_origin)

### Latent Interpolation Mosaic

In [ ]:
indeces = [30,400,1203]
LatentInterpolationMosaic(ae.encoder,
                          ae.decoder,
                          mnist_digits,
                          indeces,
                          num_row = 15,
                          num_col = 15,
                          variational = False)

In [ ]:
LatentInterpolationMosaic(va.encoder,
                          va.decoder,
                          mnist_digits,
                          indeces,
                          num_row = 15,
                          num_col = 15,
                          variational = True)